In [ ]:
import os, re
import pandas as pd
import numpy as np
import time


## Load data

In [ ]:
%%time

# Created by 02_merge_data_keywords_per_patient_with_outcomes_4.20.ipynb
all_patients0 = pd.read_csv('data/MIMIC_001_merge_patients_4.20.csv')


In [ ]:
all_patients0.columns

In [ ]:
# 41k
all_patients0.SUBJECT_ID.nunique()

## Remove discharge summary notes

In [ ]:
print(all_patients0.shape)
all_patients = all_patients0[all_patients0.CATEGORY!='Discharge summary']
print(all_patients.shape)

In [ ]:
# Dropping discharge summaries is what brings us down to only 7.7k patients.
all_patients.SUBJECT_ID.nunique()

## Find sentiment keywords

In [ ]:
#all_patients[['CHARTTIME', 'TEXT']]
# Just for comparison to find_keywords2
def find_keywords(text, keywords):
    found_keywords=[]
    text=str(text).lower()
    for i in keywords: 
        if re.search(r'\b{}\b'.format(i), text): 
            if i not in ' '.join(found_keywords):
                found_keywords.append(i)
    return [text, found_keywords]

In [ ]:
#Load keywords and combine pos+neg keywords together 
# """
all_keywords=[]
# Files are downloaded from https://drive.google.com/drive/u/0/folders/1ZdED366Wa2Hrbd2VSz7zbvRJ2iJZ1S81
# Dated November 29, 2020
neg_keywords=pd.read_csv('data-raw/negative_keywords.csv').iloc[::,0]
pos_keywords=pd.read_csv('data-raw/positive_keywords.csv').iloc[::,0]

all_keywords.append(neg_keywords.apply(lambda x: x.strip()).tolist())
all_keywords.append(pos_keywords.apply(lambda x: x.strip()).tolist())

keywords=[i.lower() for i in all_keywords for i in i]
keywords.sort(key=lambda x: len(x.split()), reverse=True)
#"""

In [ ]:
# This file is from the google drive.
"""
kw_df = pd.read_excel('data-raw/Keywords for lexicon-based sentiment classifier.xlsx',
                      sheet_name = 'Word Delineations')
kw_df.info()

# Sadly there is a space after "negative" and "positive" :/
kw_df.columns

#neg_keywords=pd.read_csv('data-raw/negative_keywords.csv').iloc[::,0]
neg_keywords = kw_df['Negative '].dropna().str.strip().str.lower().tolist()

print(neg_keywords[:5])

#pos_keywords=pd.read_csv('data-raw/positive_keywords.csv').iloc[::,0]
pos_keywords = kw_df['Positive '].dropna().str.strip().str.lower().tolist()
print(pos_keywords[:5])

kw = {'pos': pos_keywords,
            'neg': neg_keywords,
            'all': neg_keywords + pos_keywords}

# We need to have the longer phrases first to ensure that nested keywords
# are not detected multiple times. Sort by descending word count.
kw['all'].sort(key = lambda x: len(x.split()), reverse = True)
kw['all'][:5]
keywords = kw['all']
"""

In [ ]:
#Create new column of TEXT that is split per sentence 
# TODO: use a sentence segmentation algorithm rather than this simpler version.
all_patients['TEXT_SPLIT'] = all_patients.TEXT.map(lambda x: x.split('. '))

In [ ]:
from keywords import find_keywords2

In [ ]:
%%time
# This takes ~45 minutes

# TODO: convert to multicore somehow.
#all_patients['FOUND_KEYWORDS']=all_patients.TEXT_SPLIT.map(lambda x: find_keywords_final(x)[1])
all_patients['FOUND_KEYWORD_DETAILS'] = all_patients.TEXT_SPLIT.map(lambda x: find_keywords2(x, keywords))
all_patients['FOUND_KEYWORD_DETAILS'][0:10]

In [ ]:
%%time

import itertools

# Extract each keyword everytime that it occurs, and flatten the resulting nested list
all_patients['FOUND_KEYWORD_REPEATS'] = all_patients['FOUND_KEYWORD_DETAILS'].map(lambda x: list(itertools.chain.from_iterable([[key_i] * len(locations) for key_i, locations in x.items()])))
# This version doesn't count how many times the keyword occurs.
all_patients['FOUND_KEYWORD_UNIQUE'] = all_patients['FOUND_KEYWORD_DETAILS'].map(lambda x: x.keys())


In [ ]:
all_patients['FOUND_KEYWORD_REPEATS'][:10]

In [ ]:
# TBD
#all_patients['FOUND_KEYWORDS'] = all_patients.TEXT_SPLIT.map(lambda x: list(find_keywords2(x, keywords).keys()))
# all_patients['FOUND_KEYWORDS'] = all_patients.TEXT_SPLIT.map(lambda x: list(find_keywords2(x, keywords).keys()))

In [ ]:
all_patients.TEXT_SPLIT.values[0]

In [ ]:
find_keywords2(all_patients.TEXT_SPLIT.values[0], keywords)

In [ ]:
find_keywords(all_patients.TEXT_SPLIT.values[0], keywords)[1]

In [ ]:
#all_patients['FOUND_KEYWORDS']

In [ ]:
#all_patients.FOUND_KEYWORDS.map(lambda x: ', '.join(x)) 

In [ ]:
%%time

# This takes 24 minutes sadly.
# all_patients['ORIG_SENTENCE_WITH_KEYWORD']=all_patients.TEXT_SPLIT.map(lambda x: find_keywords(x, keywords)[0])

In [ ]:
#all_patients['NUM_SENTENCES_WITH_KEYWORDS']=all_patients.FOUND_KEYWORDS.map(lambda x: len(x))

In [ ]:
negative_keywords=[i.strip().lower() for i in neg_keywords]
positive_keywords=[i.strip().lower() for i in pos_keywords]

In [ ]:
print(all_patients['FOUND_KEYWORD_REPEATS'][2])
print("Negative keywords:", sum([kw in negative_keywords for kw in all_patients['FOUND_KEYWORD_REPEATS'][2]]))
print("Positive keywords:", sum([kw in positive_keywords for kw in all_patients['FOUND_KEYWORD_REPEATS'][2]]))

In [ ]:
all_patients['NUM_POS_KEYWORDS']=all_patients.FOUND_KEYWORD_REPEATS.map(lambda x: sum([kw in positive_keywords for kw in x]))
all_patients['NUM_NEG_KEYWORDS']=all_patients.FOUND_KEYWORD_REPEATS.map(lambda x: sum([kw in negative_keywords for kw in x]))

In [ ]:

#all_patients['AGGR_KEYWORDS']=all_patients.FOUND_KEYWORDS.map(lambda x: [a for a in x for a in a])

# TODO: this needs to count keywords that occur multiple times, not just once.
"""
all_patients['NUM_POS_KEYWORDS']=all_patients.FOUND_KEYWORDS.map(lambda x: len([i for i in x if i in positive_keywords]))
all_patients['NUM_NEG_KEYWORDS']=all_patients.FOUND_KEYWORDS.map(lambda x: len([i for i in x if i in negative_keywords]))
"""

In [ ]:
# This is not being used.
all_patients['FOUND_STRING_KEYWORDS']=all_patients.FOUND_KEYWORD_REPEATS.map(lambda x: ', '.join(x)) 
#final_df=pd.concat([all_patients.drop('AGGR_STRING_KEYWORDS',axis=1), all_patients['AGGR_STRING_KEYWORDS'].str.get_dummies(sep=', ')], axis=1)

In [ ]:
all_patients.head(5)
print('total number of patients:', len(all_patients.SUBJECT_ID.unique().tolist()))
print('total hospital encounters:', len(all_patients.HADM_ID.unique().tolist()))

In [ ]:
#save demographics through up to aggregate keywords (i.e. no keyword dummies)
all_patients.to_csv('data/keyword_prevalence_per_patient_nodummies_10.12.csv', index=False)

In [ ]:
all_patients.shape
# Dropped to 7.7k patients :(
all_patients.SUBJECT_ID.nunique()

In [ ]:
all_patients.head()

In [ ]:
#Subgroup of patients >=65 years old 
patients_65=all_patients[all_patients.AGE>=65]
print(patients_65.shape)
patients_65.to_csv('data/keyword_prevalence_per_patient_nodummies_age65.csv', index=False)

In [ ]:
%%time
# This takes 73 seconds.
df_kw_counts = all_patients.FOUND_KEYWORD_REPEATS.apply(pd.Series.value_counts).fillna(0).astype(int)
df_kw_counts.head()

In [ ]:
#Final_df adds columns where each keyword is a dummy variable 
#final_df=pd.concat([all_patients.drop('FOUND_STRING_KEYWORDS',axis=1),
#                    all_patients['FOUND_STRING_KEYWORDS'].str.get_dummies(sep=', ')], axis=1)

# CK: do we need to reset_index or something?
final_df = pd.concat([all_patients, df_kw_counts], axis = 1)
final_df.head(5)

In [ ]:
print('total number of patients:', len(final_df.SUBJECT_ID.unique().tolist()))
print('total hospital encounters:', len(final_df.HADM_ID.unique().tolist()))

In [ ]:
final_df.to_csv('data/keyword_prevalence_per_patient_10.12.csv', index=False)

In [ ]:
#Reproducing all the keywords used/found

In [ ]:
df=pd.read_csv('data/keyword_prevalence_per_patient_10.12.csv')

In [ ]:
negative_keywords=[i.strip().lower() for i in neg_keywords]
positive_keywords=[i.strip().lower() for i in pos_keywords]
df.FOUND_KEYWORDS.map(lambda x: len([ negative_keywords]))

In [ ]:
df.FOUND_KEYWORDS.iloc[0]

In [ ]:
[i for i in df.FOUND_KEYWORDS.iloc[0] if i in positive_keywords]

In [ ]:
just_keywords=df[['TEXT', 'FOUND_KEYWORDS']]
just_keywords.to_csv('data/list_of_keywords_found_per_note.csv', nrows=)

In [ ]:
just_keywords.shape